In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#import data
print("[USERS]")
users = pd.read_csv("C:/MachineLearning TP/recommendtationSystem/users_preprocessing.csv")
print(users.head())
print("Number of rows and cols: ", users.shape)

print("\n[GAMES]")
games = pd.read_csv("C:/MachineLearning TP/recommendtationSystem/games_preprocessing.csv")
print(games.head())
print("Number of rows and cols: ", games.shape)

[USERS]
   Unnamed: 0   user_id  products  reviews
0     1736115  11852994      54.0      6.0
1     8596407  11922584      61.0      6.0
2    10946128     19816     108.0      6.0
3     2051595  12587433      75.0      6.0
4    12077575   1627105     118.0      6.0
Number of rows and cols:  (11811106, 4)

[GAMES]
   Unnamed: 0  app_id                        title  date_release  win  mac   
0       15005  282010         Carmageddon Max Pack          1997    1    0  \
1       31261  232770                       POSTAL          1997    1    1   
2       47376      70                    Half-Life          1998    1    1   
3       34507  224920  Legacy of Kain: Soul Reaver          1999    1    0   
4        8617      20        Team Fortress Classic          1999    1    1   

   linux  rating  positive_ratio  user_reviews  price_final  price_original   
0      0      90              85           428         9.99            9.99  \
1      1      90              85          2329         0.0

In [3]:
#Encode columns in games.csv
labelEncoder = LabelEncoder()
games['rating'] = labelEncoder.fit_transform(games['rating'])
games['win'] = labelEncoder.fit_transform(games['win'])
games['mac'] = labelEncoder.fit_transform(games['mac'])

In [4]:
#train and test data
X = games.drop(columns=['app_id','rating','title','date_release'])
y = games['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [5]:
#knn
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(X_train)
neighbors = knn.kneighbors(X_test)

In [6]:
#content-based
def content_based(app_id):
    #Decision Tree Classifier
    X1 = games.drop(columns=['app_id','rating','title','date_release'])
    y1 = games['title']
    dtc = DecisionTreeClassifier()
    dtc.fit(X,y)
    #find out similar games
    index = games.index[games['app_id'] == app_id][0]
    recommended_game = dtc.predict([X1.loc[index]])[0]
    return recommended_game

In [14]:
#item-based
def item_based(app_id):
    index = games.index[games['app_id'] == app_id][0]
    _, indices = knn.kneighbors([X.loc[index]])
    game = indices.flatten()[1]
    recommended_game = games.loc[game, 'app_id']
    return recommended_game

In [15]:
#recommend games
#get app id
app_id=70

#content based
cb_recommended = content_based(app_id)
print("Recommended game by content-based filtering:", cb_recommended)

#item based
ib_recommended = item_based(app_id)
print("Recommended game by item-based filtering:", ib_recommended)
print(game)

Recommended game by content-based filtering: 8
Recommended game by item-based filtering: 1424720
30022
